

# Reporte de problemas fitosanitarios en plantaciones de agave
--------------------

## Equipo 36

| Nombre | Matr√≠cula |
| ------ | --------- |
| Andr√© Martins Cordebello | A00572928 |
| Enrique Eduardo Sol√≠s Da Costa | A00572678 |
| Delbert Francisco Custodio Vargas | A01795613 |

## Avance 3: baseline

Este avance consiste en construir un modelo de referencia que permita evaluar la viabilidad del problema. Si el baseline tiene un rendimiento similar al azar, podr√≠a indicar que el problema es intr√≠nsecamente dif√≠cil o que los datos no contienen suficiente informaci√≥n para predecir el objetivo. De lo contrario, el baseline podr√≠a como una soluci√≥n m√≠nima aceptable cuando se trabaja en escenarios donde incluso un modelo simple puede proporcionar valor.

Un baseline facilita tambi√©n la gesti√≥n de expectativas, tanto dentro del equipo como con los stakeholders, pues proporciona una comprensi√≥n inicial de lo que se puede lograr con m√©todos simples antes de invertir tiempo y recursos en enfoques m√°s complejos.

Se empieza con la carga del archivo con el cual se estar√° trabajando.

In [1]:
import pandas as pd
df = pd.read_excel('baseline.xlsx')
df.head()

C:\Users\andre\AppData\Local\Temp\ipykernel_11496\2894780301.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Unnamed: 0                   tramp_id sampling_date       lat        lon  \
0           0  167_THUE13-16-104-0009T13    2014-01-01  20.01578 -102.57259   
1           1  167_THUE13-16-104-0009T15    2014-01-01  20.01428 -102.57326   
2           2  167_THUE13-16-104-0009T25    2014-01-01  20.01524 -102.57386   
3           3  167_THUE13-16-104-0009T10    2014-01-01  20.01723 -102.57218   
4           4  167_THUE13-16-104-0009T28    2014-01-01  20.01699 -102.57360   

  municipality  square_area  plantation_age  capture_count      state  ...  \
0     VILLAMAR         0.33               0            0.0  MICHOACAN  ...   
1     VILLAMAR         0.34               0            0.0  MICHOACAN  ...   
2     VILLAMAR         0.33               0            0.0  MICHOACAN  ...   
3     VILLAMAR         0.33               0            0.0  MICHOACAN  ...   
4     VILLAMAR         0.33               0            0.0  MICHOACAN  ...   

   month_cos  critical_season  severity_encoded distance_to_nearest_hotspot  \
0   0.866025                0                 0                   62.301408   
1   0.866025                0                 0                   62.483803   
2   0.866025                0                 0                   62.407969   
3   0.866025                0                 0                   62.134520   
4   0.866025                0                 0                   62.215955   

   hotspots_within_5km                              text_feature_location  \
0                    0  La trampa con identificacion 167_THUE13-16-104...   
1                    0  La trampa con identificacion 167_THUE13-16-104...   
2                    0  La trampa con identificacion 167_THUE13-16-104...   
3                    0  La trampa con identificacion 167_THUE13-16-104...   
4                    0  La trampa con identificacion 167_THUE13-16-104...   

                                   text_feature_risk  \
0  La trampa con identificacion 167_THUE13-16-104...   
1  La trampa con identificacion 167_THUE13-16-104...   
2  La trampa con identificacion 167_THUE13-16-104...   
3  La trampa con identificacion 167_THUE13-16-104...   
4  La trampa con identificacion 167_THUE13-16-104...   

                                text_feature_capture  \
0  Durante la temporada normal el dia 1 del mes E...   
1  Durante la temporada normal el dia 1 del mes E...   
2  Durante la temporada normal el dia 1 del mes E...   
3  Durante la temporada normal el dia 1 del mes E...   
4  Durante la temporada normal el dia 1 del mes E...   

                             text_feature_plantation  \
0  En una plantacion de agave azul con area de 0....   
1  En una plantacion de agave azul con area de 0....   
2  En una plantacion de agave azul con area de 0....   
3  En una plantacion de agave azul con area de 0....   
4  En una plantacion de agave azul con area de 0....   

                             text_feature_all_things  
0  En una plantacion de agave azul con area de 0....  
1  En una plantacion de agave azul con area de 0....  
2  En una plantacion de agave azul con area de 0....  
3  En una plantacion de agave azul con area de 0....  
4  En una plantacion de agave azul con area de 0....  

[5 rows x 31 columns]

Para lograr realizar un estudio de manera r√°pida y concisa, se estar√° trabajando √∫nicamente con los datos obtenidos en los a√±os 2023 a 2025 para reducir tiempos de entrenamiento y poder probar con m√°s modelos en menos tiempo. Adem√°s, se eliminan columnas de texto que se utilizar√°n para entrenar el LLM del chatbot que se desarrollar√°, pero que no son de importancia para este estudio.

In [2]:
# ===============================
# Filtrar dataset (2023-2025) y eliminar columnas no necesarias
# ===============================

# Filtrar solo los a√±os 2023, 2024 y 2025
df = df[df['year'].between(2023, 2025)]

# Verificar los valores √∫nicos en la columna 'year'
print("A√±os disponibles en el dataset:", df['year'].unique())

# Eliminar columnas de texto irrelevantes
cols_to_drop = [
    'text_feature_location',
    'text_feature_risk',
    'text_feature_capture',
    'text_feature_plantation',
    'text_feature_all_things'
]

# Verificar si existen antes de eliminarlas
cols_present = [col for col in cols_to_drop if col in df.columns]
df = df.drop(columns=cols_present)

print(f"Columnas eliminadas: {cols_present}")
print("Dimensiones finales del dataset:", df.shape)

# (Opcional) Guardar el dataset filtrado en un nuevo archivo
df.to_excel("baseline_filt_23_25.xlsx", index=False)
print("Archivo guardado como baseline_filt_23_25.xlsx")


A√±os disponibles en el dataset: [2023 2024 2025]
Columnas eliminadas: ['text_feature_location', 'text_feature_risk', 'text_feature_capture', 'text_feature_plantation', 'text_feature_all_things']
Dimensiones finales del dataset: (171683, 26)
Archivo guardado como baseline_filt_23_25.xlsx


Se vuelve a hacer un an√°lisis de las variables categoricas para determinar si son relevantes para el estudio y si necesitan alg√∫n tipo de encoder.

In [3]:
# ===============================
# An√°lisis inicial del dataset filtrado (2023-2025)
# ===============================

import pandas as pd

# Cargar el archivo filtrado
df = pd.read_excel("baseline_filt_23_25.xlsx")

# Mostrar informaci√≥n general
print("Dimensiones del dataset:", df.shape)
print("\nColumnas del dataset:")
print(df.columns.tolist())

# Tipos de datos y conteo de valores nulos
print("\nResumen de tipos de datos y valores nulos:")
print(df.info())
print("\nValores nulos por columna:")
print(df.isnull().sum())

# Verificar si hay variables categ√≥ricas
print("\nVariables categ√≥ricas detectadas:")
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print(cat_cols if len(cat_cols) > 0 else "No se detectaron variables categ√≥ricas")

# Mostrar ejemplo de los primeros registros
print("\nPrimeras filas del dataset:")
display(df.head())

Dimensiones del dataset: (171683, 26)

Columnas del dataset:
['Unnamed: 0', 'tramp_id', 'sampling_date', 'lat', 'lon', 'municipality', 'square_area', 'plantation_age', 'capture_count', 'state', 'square_area_imputed', 'month', 'year', 'year_month', 'day_of_year_sin', 'day_of_year_cos', 'day_of_week_sin', 'day_of_week_cos', 'week_of_year_sin', 'week_of_year_cos', 'month_sin', 'month_cos', 'critical_season', 'severity_encoded', 'distance_to_nearest_hotspot', 'hotspots_within_5km']

Resumen de tipos de datos y valores nulos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171683 entries, 0 to 171682
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   Unnamed: 0                   171683 non-null  int64         
 1   tramp_id                     171683 non-null  object        
 2   sampling_date                171683 non-null  datetime64[ns]
 3   lat                        

Unnamed: 0                   tramp_id sampling_date        lat         lon  \
0      656173  162_THUE13-16-074-0027T01    2023-01-01  20.096520 -102.814320   
1      656174  162_THUE13-16-074-0007T01    2023-01-01  20.147118 -102.896716   
2      656175  162_THUE13-16-074-0013T01    2023-01-01  20.142147 -102.882885   
3      656176  162_THUE13-16-074-0012T01    2023-01-01  20.144013 -102.882413   
4      656177  162_THUE13-16-074-0024T01    2023-01-01  20.146415 -102.881007   

            municipality  square_area  plantation_age  capture_count  \
0  COJUMATLAN DE REGULES          1.0               0            0.0   
1  COJUMATLAN DE REGULES          1.0               0            0.0   
2  COJUMATLAN DE REGULES          2.0               0            0.0   
3  COJUMATLAN DE REGULES          2.0               0            0.0   
4  COJUMATLAN DE REGULES          2.0               0            0.0   

       state  ...  day_of_week_sin  day_of_week_cos  week_of_year_sin  \
0  MICHOACAN  ...        -0.781831          0.62349     -2.449294e-16   
1  MICHOACAN  ...        -0.781831          0.62349     -2.449294e-16   
2  MICHOACAN  ...        -0.781831          0.62349     -2.449294e-16   
3  MICHOACAN  ...        -0.781831          0.62349     -2.449294e-16   
4  MICHOACAN  ...        -0.781831          0.62349     -2.449294e-16   

  week_of_year_cos  month_sin  month_cos  critical_season  severity_encoded  \
0              1.0        0.5   0.866025                0                 0   
1              1.0        0.5   0.866025                0                 0   
2              1.0        0.5   0.866025                0                 0   
3              1.0        0.5   0.866025                0                 0   
4              1.0        0.5   0.866025                0                 0   

   distance_to_nearest_hotspot  hotspots_within_5km  
0                       9999.0                    0  
1                       9999.0                    0  
2                       9999.0                    0  
3                       9999.0                    0  
4                       9999.0                    0  

[5 rows x 26 columns]

Se opt√≥ por eliminar la columna 'tramp_ID', ya que esta no agrega datos relevantes, as√≠ como severity_encoded que es redundante con capture_count, as√≠ como codificar las columnas 'municipality' y 'state' y separar la columna 'year_month' para poder obtener √∫nicamente el mes (ya existe una columna con el a√±o del dato).

In [4]:
# ===============================
# Preparaci√≥n de variables categ√≥ricas
# ===============================

# Eliminar ID de trampa (no es informativo para el efecto de este modelo)
if 'tramp_id' in df.columns:
    df = df.drop(columns=['tramp_id'])

#Eliminar severity_encoded (parece redundante con capture_count)
if 'severity_encoded' in df.columns:
    df = df.drop(columns=['severity_encoded'])

# Convertir columna 'year_month' a variables num√©ricas separadas y eliminar year (ya existente)
if 'year_month' and 'sampling_date' in df.columns:
    df['day'] = df['sampling_date'].dt.day
    df['month'] = df['sampling_date'].dt.month
    df = df.drop(columns=['year_month','sampling_date'])

# Codificar variables categ√≥ricas relevantes ('municipality', 'state')
df = pd.get_dummies(df, columns=['municipality', 'state'], drop_first=True)

# Verificar resultado
print("Columnas finales despu√©s de la codificaci√≥n:")
print(df.columns.tolist())

Columnas finales despu√©s de la codificaci√≥n:
['Unnamed: 0', 'lat', 'lon', 'square_area', 'plantation_age', 'capture_count', 'square_area_imputed', 'month', 'year', 'day_of_year_sin', 'day_of_year_cos', 'day_of_week_sin', 'day_of_week_cos', 'week_of_year_sin', 'week_of_year_cos', 'month_sin', 'month_cos', 'critical_season', 'distance_to_nearest_hotspot', 'hotspots_within_5km', 'day', 'municipality_ACATIC', 'municipality_ACATLAN DE JUAREZ', 'municipality_AHUACATLAN', 'municipality_AHUALULCO DE MERCADO', 'municipality_AMACUECA', 'municipality_AMATITAN', 'municipality_AMATLAN DE CANAS', 'municipality_AMECA', 'municipality_ARANDAS', 'municipality_ATENGO', 'municipality_ATENGUILLO', 'municipality_ATOTONILCO EL ALTO', 'municipality_ATOYAC', 'municipality_AUTLAN DE NAVARRO', 'municipality_AYOTLAN', 'municipality_AYUTLA', 'municipality_BRISENAS', 'municipality_CANADAS DE OBREGON', 'municipality_CASIMIRO CASTILLO', 'municipality_CHAPALA', 'municipality_CHIQUILISTLAN', 'municipality_CHURINTZIO'

Se hace la separaci√≥n de la base de datos en variables dependientes e independientes para que se pueda empezar a crear modelos de predicci√≥n.

In [5]:
# Definir variables predictoras y objetivo
X = df.drop(columns=['capture_count'])
y = df['capture_count']

print("\nX shape:", X.shape)
print("y shape:", y.shape)



X shape: (171683, 167)
y shape: (171683,)


Ya que hizo la separaci√≥n de las variables X y Y, se procede a crear un primer modelo de predicci√≥n utilizando Random Forest Regressor. Se opt√≥ por utilizar este m√©todo por que trabaja bien con datos no lineales, tolera outliers y nos permitir√° obtener la importancia de las variables. Se realiza el estudio utilizando los valores default de RandomForestRegressor.

In [6]:
# ===============================
# Modelo Baseline - Random Forest
# ===============================

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Dividir dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Entrenar modelo baseline
rf = RandomForestRegressor(
    n_estimators=100,       # n√∫mero de √°rboles
    max_depth=None,         # sin l√≠mite de profundidad
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

# Predicciones
y_pred = rf.predict(X_test)

# Evaluaci√≥n
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Evaluaci√≥n del modelo baseline con Random Forest Regressor:")
print(f"MAE  = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")
print(f"R¬≤   = {r2:.4f}")


Evaluaci√≥n del modelo baseline con Random Forest Regressor:
MAE  = 2.8432
RMSE = 5.3918
R¬≤   = 0.4908


In [8]:
# =========================================
# Baseline con XGBoost
# - Modelo 1: reg:squarederror (MSE)
# - Modelo 2: count:poisson (para conteos >= 0)
# =========================================

from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# --- split para nuevo estudio
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

def eval_reg(y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2   = r2_score(y_true, y_pred)
    return mae, rmse, r2

# ============== Modelo 1: cuadr√°tico est√°ndar ==============
xgb_mse = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1
)

xgb_mse.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    early_stopping_rounds=50,
    verbose=False
)

pred_mse = xgb_mse.predict(X_valid)
mae1, rmse1, r21 = eval_reg(y_valid, pred_mse)

print("üîπ XGBoost (reg:squarederror)")
print(f"MAE  : {mae1:.3f}")
print(f"RMSE : {rmse1:.3f}")
print(f"R¬≤   : {r21:.3f}")
print(f"Best iteration (early stop): {xgb_mse.best_iteration}")

# ============== Modelo 2: Poisson (siempre que y >= 0) ==============
if (y_train.min() >= 0) and (y_valid.min() >= 0):
    xgb_pois = XGBRegressor(
        objective='count:poisson',      # mejor para conteos
        eval_metric='poisson-nloglik',  # m√©trica de validaci√≥n
        n_estimators=800,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1
    )

    xgb_pois.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        early_stopping_rounds=50,
        verbose=False
    )

    pred_pois = xgb_pois.predict(X_valid)
    mae2, rmse2, r22 = eval_reg(y_valid, pred_pois)

    print("\nüîπ XGBoost (count:poisson)")
    print(f"MAE  : {mae2:.3f}")
    print(f"RMSE : {rmse2:.3f}")
    print(f"R¬≤   : {r22:.3f}")
    print(f"Best n_estimators (early stop): {xgb_pois.best_iteration}")
else:
    print("\n  Omitido modelo Poisson porque existen valores negativos en y.")

# ============== Importancias de variables (top 15) ==============
model_to_plot = xgb_pois if 'xgb_pois' in locals() else xgb_mse
importances = pd.Series(model_to_plot.feature_importances_, index=X.columns)
top15 = importances.sort_values(ascending=False).head(15)
print("\nTop 15 features:")
print(top15)

plt.figure(figsize=(8,6))
top15.sort_values().plot(kind='barh')
plt.title("XGBoost - Importancia de variables (top 15)")
plt.tight_layout()
plt.show()


üîπ XGBoost (reg:squarederror)
MAE  : 3.136
RMSE : 5.580
R¬≤   : 0.455
Best iteration (early stop): 498

üîπ XGBoost (count:poisson)
MAE  : 3.134
RMSE : 5.636
R¬≤   : 0.444
Best n_estimators (early stop): 799

Top 15 features:
hotspots_within_5km                         0.075826
state_NAYARIT                               0.037307
municipality_EL GRULLO                      0.024916
distance_to_nearest_hotspot                 0.023550
municipality_IXTLAHUACAN DEL RIO            0.021845
state_JALISCO                               0.020987
municipality_VILLAMAR                       0.020902
year                                        0.019723
municipality_JUCHITLAN                      0.016684
municipality_TUXCACUESCO                    0.016171
municipality_YAHUALICA DE GONZALEZ GALLO    0.015818
municipality_ROMITA                         0.015632
municipality_MARAVATIO                      0.015345
municipality_AUTLAN DE NAVARRO              0.014855
municipality_CANADAS DE OBREG

Al realizar los estudios con XGBoost y con Random Forest Regressor, encontramos resultados ...............

In [9]:
# ==============================================
# Bayesian Optimization (Optuna) + Random Forest
# Optuna "Fast Mode" para RandomForestRegressor
# - Muestra opcional de filas
# - Hold-out en objetivo (sin CV)
# - Pruner para cortar trials malos
# - Espacio de b√∫squeda estrecho
# ==============================================
import optuna


# ---------- Toggles ----------
ROWS_SAMPLE   = 60000      # None para usar todo; pon 60_000 ~ 120_000 seg√∫n tu m√°quina
TEST_SIZE_OBJ = 0.2        # hold-out dentro del objetivo
TEST_SIZE_FIN = 0.2        # hold-out final para reporte
N_TRIALS      = 20         # n interacciones
TIMEOUT       = None       # en segundos; p.ej. 900 para 15 min
RANDOM_STATE  = 42

# ---------- Copia y casting a float32 (m√°s r√°pido/memoria) ----------
X_bo = X.copy()
for c in X_bo.columns:
    if np.issubdtype(X_bo[c].dtype, np.floating):
        X_bo[c] = X_bo[c].astype(np.float32)

y_bo = y.astype(np.float32)

# ---------- Muestreo opcional ----------
if ROWS_SAMPLE is not None and ROWS_SAMPLE < len(X_bo):
    X_bo, _, y_bo, _ = train_test_split(
        X_bo, y_bo, train_size=ROWS_SAMPLE, random_state=RANDOM_STATE
    )

# ---------- Split fijo para el objetivo (repetible y r√°pido) ----------
Xtr_obj, Xva_obj, ytr_obj, yva_obj = train_test_split(
    X_bo, y_bo, test_size=TEST_SIZE_OBJ, random_state=RANDOM_STATE
)

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 250, 320),  # tope moderado
        "max_depth": trial.suggest_categorical("max_depth", [None, 8, 12, 24]),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 12),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 8),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", 0.7, 1.0]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "n_jobs": -1,
        "random_state": RANDOM_STATE,
    }

    # Entrena en hold-out (m√°s r√°pido que CV)
    model = RandomForestRegressor(**params)
    model.fit(Xtr_obj, ytr_obj)

    pred = model.predict(Xva_obj)
    rmse = mean_squared_error(yva_obj, pred, squared=False)

    # --- Pruning: si va mal, corta temprano comparando con mediana hist√≥rica
    trial.report(rmse, step=0)
    if trial.should_prune():
        raise optuna.TrialPruned()

    return rmse

# ---------- Pruner + estudio ----------
pruner = optuna.pruners.MedianPruner(n_warmup_steps=5)
study = optuna.create_study(direction="minimize", pruner=pruner)
study.optimize(objective, n_trials=N_TRIALS, timeout=TIMEOUT, show_progress_bar=True)

print("\nüèÜ Mejores hiperpar√°metros (fast mode):")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")
print(f"Mejor RMSE (hold-out objetivo): {study.best_value:.4f}")

# ---------- Re-entrenar modelo final con TODO el dataset ----------
best_params = study.best_params.copy()
best_params.update({"n_jobs": -1, "random_state": RANDOM_STATE})

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE_FIN, random_state=RANDOM_STATE
)

rf_best = RandomForestRegressor(**best_params)
rf_best.fit(X_train, y_train)
y_pred = rf_best.predict(X_test)

mae  = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2   = r2_score(y_test, y_pred)

print("\nüìä Desempe√±o final (hold-out):")
print(f"MAE  : {mae:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"R¬≤   : {r2:.4f}")



C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-10-12 18:39:03,554] A new study created in memory with name: no-name-15f062da-a423-4ed5-a134-bae1eb9614f6
Best trial: 0. Best value: 5.81893:   5%|‚ñå         | 1/20 [00:07<02:25,  7.64s/it]

[I 2025-10-12 18:39:11,193] Trial 0 finished with value: 5.8189293847269825 and parameters: {'n_estimators': 290, 'max_depth': 24, 'min_samples_split': 6, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 5.8189293847269825.


Best trial: 0. Best value: 5.81893:  10%|‚ñà         | 2/20 [00:23<03:45, 12.50s/it]

[I 2025-10-12 18:39:27,106] Trial 1 finished with value: 5.953514730165431 and parameters: {'n_estimators': 270, 'max_depth': 8, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 0.7, 'bootstrap': True}. Best is trial 0 with value: 5.8189293847269825.


Best trial: 0. Best value: 5.81893:  15%|‚ñà‚ñå        | 3/20 [01:05<07:23, 26.09s/it]

...

[I 2025-10-12 18:47:20,542] Trial 17 finished with value: 5.953452968489629 and parameters: {'n_estimators': 284, 'max_depth': None, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': 'log2', 'bootstrap': True}. Best is trial 15 with value: 5.469410304506807.


Best trial: 15. Best value: 5.46941:  95%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñå| 19/20 [08:56<00:28, 28.07s/it]

[I 2025-10-12 18:48:00,315] Trial 18 finished with value: 5.742236105204214 and parameters: {'n_estimators': 299, 'max_depth': 12, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 1.0, 'bootstrap': True}. Best is trial 15 with value: 5.469410304506807.


Best trial: 15. Best value: 5.46941: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 20/20 [09:38<00:00, 28.92s/it]


[I 2025-10-12 18:48:42,039] Trial 19 finished with value: 5.484474417986437 and parameters: {'n_estimators': 284, 'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': 0.7, 'bootstrap': True}. Best is trial 15 with value: 5.469410304506807.

üèÜ Mejores hiperpar√°metros (fast mode):
  n_estimators: 283
  max_depth: None
  min_samples_split: 8
  min_samples_leaf: 3
  max_features: 0.7
  bootstrap: True
Mejor RMSE (hold-out objetivo): 5.4694

üìä Desempe√±o final (hold-out):
MAE  : 2.7824
RMSE : 5.1761
R¬≤   : 0.5307


In [11]:
# ============================================================
# Optuna "Fast Mode" para XGBoost (MSE y Poisson) + reporte
# - Muestra opcional de filas
# - Hold-out dentro del objetivo (sin CV) + early stopping
# - Pruner para cortar trials malos
# - Espacio de b√∫squeda razonable (r√°pido)
# Requisitos: pip install optuna xgboost
# ============================================================

# ----------------- Toggles -----------------
ROWS_SAMPLE   = 60000   # None para usar todo (puedes bajar/subir)
TEST_SIZE_OBJ = 0.2      # hold-out interno del objetivo (BO)
TEST_SIZE_FIN = 0.2      # hold-out final para reporte
N_TRIALS      = 40       # sube cuando quieras (80‚Äì150 si tienes tiempo)
TIMEOUT       = 1200     # p.ej. 900 (15 min)
RANDOM_STATE  = 42
#TREE_METHOD   = "gpu_hist"   # "hist" CPU r√°pido; si tienes GPU: "gpu_hist"
VERBOSE_BOOST = False    # True para ver el training de cada trial
EARLY_STOP    = 50       # early stopping rounds
DEVICE        = "cuda"

# ----------------- Copia + casting (memoria/velocidad) -----------------
X_bo = X.copy()
for c in X_bo.columns:
    if np.issubdtype(X_bo[c].dtype, np.floating):
        X_bo[c] = X_bo[c].astype(np.float32)
y_bo = y.astype(np.float32)

# ----------------- Muestreo opcional -----------------
if ROWS_SAMPLE is not None and ROWS_SAMPLE < len(X_bo):
    X_bo, _, y_bo, _ = train_test_split(X_bo, y_bo, train_size=ROWS_SAMPLE, random_state=RANDOM_STATE)

# ----------------- Split fijo para la funci√≥n objetivo -----------------
Xtr_obj, Xva_obj, ytr_obj, yva_obj = train_test_split(
    X_bo, y_bo, test_size=TEST_SIZE_OBJ, random_state=RANDOM_STATE
)

def make_objective(objective_name: str):
    """Crea una funci√≥n objetivo de Optuna para XGBRegressor."""
    assert objective_name in {"mse", "poisson"}
    use_poisson = (objective_name == "poisson")

    def objective(trial):
        params = {
            "objective": "count:poisson" if use_poisson else "reg:squarederror",
            "eval_metric": "poisson-nloglik" if use_poisson else "rmse",
            #"tree_method": TREE_METHOD,
            "n_estimators": trial.suggest_int("n_estimators", 200, 700),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "gamma": trial.suggest_float("gamma", 0.0, 5.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0, log=True),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
            "random_state": RANDOM_STATE,
            "n_jobs": -1,
            "device": DEVICE
        }

        model = XGBRegressor(**params)
        model.fit(
            Xtr_obj, ytr_obj,
            eval_set=[(Xva_obj, yva_obj)],
            early_stopping_rounds=EARLY_STOP,
            verbose=VERBOSE_BOOST
        )

        # Usamos RMSE como m√©trica objetivo (aun en Poisson para comparabilidad)
        pred = model.predict(Xva_obj)
        rmse = mean_squared_error(yva_obj, pred, squared=False)

        trial.report(rmse, step=0)
        if trial.should_prune():
            raise optuna.TrialPruned()

        return rmse

    return objective

def run_bo_for(objective_name: str):
    pruner = optuna.pruners.MedianPruner(n_warmup_steps=5)
    study = optuna.create_study(direction="minimize", pruner=pruner)
    study.optimize(make_objective(objective_name), n_trials=N_TRIALS, timeout=TIMEOUT, show_progress_bar=True)
    return study

# ----------------- Ejecutar BO para MSE -----------------
print("\n================ XGBoost (reg:squarederror) ================")
study_mse = run_bo_for("mse")
print("Mejores hiperpar√°metros (MSE):")
for k, v in study_mse.best_params.items():
    print(f"  {k}: {v}")
print(f"Mejor RMSE (hold-out objetivo): {study_mse.best_value:.4f}")

# ----------------- Ejecutar BO para Poisson (si procede) -----------------
can_poisson = (y_bo.min() >= 0)
study_pois = None
if can_poisson:
    print("\n================ XGBoost (count:poisson) ================")
    study_pois = run_bo_for("poisson")
    print("Mejores hiperpar√°metros (Poisson):")
    for k, v in study_pois.best_params.items():
        print(f"  {k}: {v}")
    print(f"Mejor RMSE (hold-out objetivo): {study_pois.best_value:.4f}")
else:
    print("\n  Poisson omitido: existen valores negativos en y.")

# ----------------- Re-entrenar y evaluar en hold-out final -----------------
def evaluate_best(study, label: str):
    best_params = study.best_params.copy()
    best_params.update({
        "objective": "count:poisson" if label == "Poisson" else "reg:squarederror",
        "eval_metric": "poisson-nloglik" if label == "Poisson" else "rmse",
        #"tree_method": TREE_METHOD,
        "random_state": RANDOM_STATE,
        "n_jobs": -1,
        "device": DEVICE
    })

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE_FIN, random_state=RANDOM_STATE
    )

    model = XGBRegressor(**best_params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        early_stopping_rounds=EARLY_STOP,
        verbose=VERBOSE_BOOST
    )

    pred = model.predict(X_test)
    mae  = mean_absolute_error(y_test, pred)
    rmse = mean_squared_error(y_test, pred, squared=False)
    r2   = r2_score(y_test, pred)

    print(f"\nüìä Desempe√±o final ({label})")
    print(f"MAE  : {mae:.4f}")
    print(f"RMSE : {rmse:.4f}")
    print(f"R¬≤   : {r2:.4f}")

    # Importancias top-15
    importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False).head(15)
    print("\nüîù Top-15 importancias de variables:")
    print(importances)

# Evaluar ambos (y compara)
evaluate_best(study_mse, "MSE")
if study_pois is not None:
    evaluate_best(study_pois, "Poisson")

[I 2025-10-12 18:51:35,982] A new study created in memory with name: no-name-7e9a6fa0-d7f6-4e25-b6ba-21eb9e14e498



================ XGBoost (reg:squarederror) ================


Best trial: 0. Best value: 5.77405:   2%|‚ñé         | 1/40 [00:04<03:08,  4.83s/it, 4.82/1200 seconds]

[I 2025-10-12 18:51:40,806] Trial 0 finished with value: 5.7740478515625 and parameters: {'n_estimators': 298, 'learning_rate': 0.09989358028731785, 'max_depth': 10, 'min_child_weight': 9, 'subsample': 0.636132149448843, 'colsample_bytree': 0.7594959246568643, 'gamma': 4.013320430830533, 'reg_lambda': 0.7446019619131931, 'reg_alpha': 0.6228460104531647}. Best is trial 0 with value: 5.7740478515625.


Best trial: 0. Best value: 5.77405:   5%|‚ñå         | 2/40 [00:07<02:15,  3.57s/it, 7.51/1200 seconds]

...

[I 2025-10-12 19:02:33,400] Trial 37 finished with value: 5.747844696044922 and parameters: {'n_estimators': 598, 'learning_rate': 0.10337237814463064, 'max_depth': 10, 'min_child_weight': 2, 'subsample': 0.6793482554429016, 'colsample_bytree': 0.5755720468108596, 'gamma': 3.409212893086518, 'reg_lambda': 4.319611831140319, 'reg_alpha': 0.714253247796055}. Best is trial 12 with value: 5.675923824310303.


Best trial: 12. Best value: 5.67592:  98%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñä| 39/40 [05:48<00:09,  9.29s/it, 348.85/1200 seconds]

[I 2025-10-12 19:02:41,577] Trial 38 finished with value: 5.730019569396973 and parameters: {'n_estimators': 558, 'learning_rate': 0.049940216546035206, 'max_depth': 9, 'min_child_weight': 4, 'subsample': 0.9177375174370561, 'colsample_bytree': 0.6624932895912289, 'gamma': 4.637926978252883, 'reg_lambda': 5.051524290474587, 'reg_alpha': 0.8254315976266839}. Best is trial 12 with value: 5.675923824310303.


Best trial: 12. Best value: 5.67592: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 40/40 [05:58<00:00,  8.97s/it, 358.60/1200 seconds]


[I 2025-10-12 19:02:51,333] Trial 39 finished with value: 5.734367370605469 and parameters: {'n_estimators': 508, 'learning_rate': 0.06321499980979121, 'max_depth': 11, 'min_child_weight': 3, 'subsample': 0.7429872285834358, 'colsample_bytree': 0.6903606964384087, 'gamma': 4.112540785796363, 'reg_lambda': 1.3858572415823527, 'reg_alpha': 0.9338703584141843}. Best is trial 12 with value: 5.675923824310303.
Mejores hiperpar√°metros (Poisson):
  n_estimators: 700
  learning_rate: 0.05099090693845832
  max_depth: 9
  min_child_weight: 10
  subsample: 0.9735393163169309
  colsample_bytree: 0.6115204014242451
  gamma: 0.08504394724660294
  reg_lambda: 0.10036601436075619
  reg_alpha: 0.07067120154248302
Mejor RMSE (hold-out objetivo): 5.6759

üìä Desempe√±o final (MSE)
MAE  : 2.9077
RMSE : 5.3732
R¬≤   : 0.4943

üîù Top-15 importancias de variables:
hotspots_within_5km                 0.058601
municipality_VILLAMAR               0.026070
municipality_IXTLAHUACAN DEL RIO    0.025644
state_N

**Observaciones generales:**

- Se utilizaron RandomForestRegressor y XGBoost debido a que se trata de un sistema no lineal con factores que tienen que ver con series de tiempo.
- Antes de aplicar Bayesian Optimization, el modelo con RandomForestRegressor tuvo un mejor performance que XGBoost.
- Al aplicar el Bayesian Optimization, todos los modelos mejoraron.
- Bayesian optimization se trata de un buscador de mejores hiperparametros, el cual hace que el c√≥digo tarde mucho m√°s en ejecutarse.
- Despu√©s del Bayesian Optimization el Random Forest se mantiene como el m√°s robusto, por que:
    - Logra el mejor equilibrio entre sesgo y varianza.
    - Parece modelar mejor las interacciones no lineales entre ubicaci√≥n, a√±o y condiciones.
    - Adem√°s, es menos sensible al tuning extremo que XGBoost, por lo que ofrece buena estabilidad para el baseline final.
    - XGBoost MSE ofrece resultados casi id√©nticos al RF, lo cual es una buena se√±al de que los patrones subyacentes est√°n bien capturados.
    - Puede superar al RF si se ampl√≠a la b√∫squeda de hiperpar√°metros (m√°s n_trials o ajustes finos de learning_rate y subsample).
    - XGBoost Poisson no mejora el rendimiento, pese a ser el enfoque te√≥ricamente m√°s adecuado para variables de conteo.
    - Esto sugiere que la variable capture_count no sigue una distribuci√≥n Poisson pura (quiz√°s hay sobredispersi√≥n o ruido espacial que el modelo no logra ajustar).